In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<center>“<b>The most exciting rhythms seem unexpected and complex, the most beautiful melodies simple and inevitable.</b>”<br><t>― W.H. Auden, The Complete Works of W.H. Auden: Prose, Volume III, 1949-1955</center>

In [ ]:
# music genre Classification
# Goal of this project is to apply the concepts learned in CNN( / convolutional Neural Network)
# here I m using GTZAN dataset 

We have genres :
* Rock
* Clasical
* Disco
* Hip-Hop
* Jazz
* Blues
* Metal
* Pop
* Country
* Reggae (hearing this first time :P)

Total 10

In [ ]:
# lets import libraries that we will be usings
# we already have imported pandas and numpy

import matplotlib.pyplot as plt
import scipy
import librosa
import librosa.display
import IPython.display as ipd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
import pickle # model pickling for future use

# Loading Datasets

In [ ]:
# Loading dataset
# we have 2 CSVs here, one containing features for 30 sec audio file, mean & variance for diff features we have, then
# and one for 3 sec audio files. I will be using 3 sec audio
dataf = pd.read_csv("../input/gtzan-dataset-music-genre-classification/Data/features_3_sec.csv")
dataf.head()

In [ ]:
dataf.tail()

In [ ]:
dataf.shape

In [ ]:
dataf.describe()

In [ ]:
# removing filename column
dataf = dataf.drop(labels='filename',axis=1)

In [ ]:
# looking into what i have 
sample_audio = "../input/gtzan-dataset-music-genre-classification/Data/genres_original/pop/pop.00003.wav"
sample, sample_rate = librosa.load(sample_audio)


ipd.Audio(sample, rate=sample_rate)

In [ ]:
sample, sample_rate = librosa.load(sample_audio, sr=16000)
print(len(sample),sample_rate)

In [ ]:
print(type(sample),sample_rate)
sample, sample_rate = librosa.load(sample_audio, sr=16000)

# plotting raw wave Files, here it is for Pop genre
fig = plt.figure(figsize=(14,6))
ax1 = fig.add_subplot(211)
ax1.set_title("../input/gtzan-dataset-music-genre-classification/Data/genres_original/pop/pop.00003.wav")
ax1.set_xlabel('time')
ax1.set_ylabel('Amptitude')
librosa.display.waveplot(sample)
plt.show()

In [ ]:
# Spectrogram, we have this in images folder
# way of representing signal loudness at diff freq
# also known as Sonographs
# when data is 3D then waterfall

stft = librosa.stft(sample)
stft_db = librosa.amplitude_to_db(abs(stft))
plt.figure(figsize=(14,6))
librosa.display.specshow(stft,sr=sample_rate,x_axis='time',y_axis='hz')
plt.colorbar()

In [ ]:
stft = librosa.stft(sample)
stft_db = librosa.amplitude_to_db(abs(stft))
plt.figure(figsize=(14,6))
librosa.display.specshow(stft_db,sr=sample_rate,x_axis='time',y_axis='hz')
plt.colorbar()

In [ ]:
# Rolloff - feq below which a specified percentage of the total spectral lies / 85%
from sklearn.preprocessing import normalize

spectral_rolloff = librosa.feature.spectral_rolloff(sample+0.01,sr=sample_rate)[0]
plt.figure(figsize=(14,6))
librosa.display.waveplot(sample,sr=sample_rate,alpha=0.3)

In [ ]:
# Zero crossing 
plt.figure(figsize=(14,6))
plt.plot(sample[8000:12000])
plt.grid()

In [ ]:
#count
zero_cross = librosa.zero_crossings(sample[8000:12000],pad=False)
print("Count {}".format(sum(zero_cross)))

# Feature Extraction

In [ ]:
class_list = dataf.iloc[:,-1] 
convert = LabelEncoder()

In [ ]:
y = convert.fit_transform(class_list)
y

In [ ]:
dataf.iloc[:,:-1]

In [ ]:
# scaling features
from sklearn.preprocessing import StandardScaler
fit = StandardScaler()
X = fit.fit_transform(np.array(dataf.iloc[:,:-1],dtype=float))

In [ ]:
# dividing into training and test Data
X_train,x_test, Y_train, y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
print(len(Y_train),len(y_test))

# Model Building

In [ ]:
# Using CNN algorithm
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
# building model

model = Sequential()

model.add(Dense(512,input_shape=(X_train.shape[1],),activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(256,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(10,activation='softmax'))


model.summary()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics='accuracy')

In [ ]:
earlystop = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=10,min_delta=0.0001)
modelcheck = ModelCheckpoint('best_model.hdf5',monitor='val_accuracy',verbose=1,save_best_only=True,mode='max')

In [ ]:
history = model.fit(X_train,Y_train, validation_data=(x_test,y_test), epochs=600, callbacks=[earlystop,modelcheck], batch_size=128)

In [ ]:
from matplotlib import pyplot 
pyplot.plot(history.history['loss'], label='train') 
pyplot.plot(history.history['val_loss'], label='test') 
pyplot.legend()
pyplot.show()

In [ ]:
#from keras.models import load_model
#Model = load_model("./best_model.hdf5")

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test,y_test,batch_size=128)
print("Test loss : ",test_loss)
print("\nBest test accuracy : ",test_accuracy*100)